In [2]:
#%pip install opencv-python

In [3]:
!pip install pytesseract
!pip install imutils

In [4]:
import os
import time
import pickle
import cv2
import pytesseract
import numpy as np  
from keras import applications
from keras.applications.inception_v3 import preprocess_input
from keras.utils.image_utils import img_to_array
from keras.models import Model
import re
import cv2
from imutils import contours
import random
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    GPT2PreTrainedModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

C:\Users\naveen.muralidharan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def extract_text_from_roi(frame_count, x, y, w, h, img):
    roi = img[y:y+h, x:x+w]
    cv2.imwrite(f'temp/extracted_{frame_count}.jpg', roi)
        # Convert ROI to grayscale
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    """ # Apply adaptive thresholding
    roi_threshold = cv2.adaptiveThreshold(roi_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
         # Perform morphological operations (e.g., erosion and dilation)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    roi_processed = cv2.morphologyEx(roi_threshold, cv2.MORPH_CLOSE, kernel)"""
      
    text = pytesseract.image_to_string(roi_gray)   
    text = re.sub(r'\d', '', text) 
    if '\n' in text:
        first_line = text.split('\n')[0]
    else:
        first_line = text
    #print(first_line)
    #print(first_line)
    return first_line

In [6]:
def getScore(x, y, w, h, img):
    keywords = {'1', '2', '3', '4', '6', 'WD', 'W'}
    roi = img[y:y+h, x:x+w]
    gray_image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray_image)# Preprocess the grayscale image (optional)
    print(text)
    return text

In [7]:
def GenerateCommentarybasedonEvent(model_dir, prompt):

    model = GPT2LMHeadModel.from_pretrained(model_dir)
    tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
    commentary_string =''
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    sample_outputs = model.generate(
                        input_ids= input_ids,
                        do_sample = True,
                        #num_beams= 5,
                        max_length = 100,
                        top_k = 50,
                        top_p=0.85, 
                        num_return_sequences=1
                    )

    print("Output:\n" + 100 * '-')
    for i, sample_output in enumerate(sample_outputs):
          print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
          commentary_string = commentary_string +"." +tokenizer.decode(sample_output, skip_special_tokens=True)

    return commentary_string

In [8]:
def generateCommentary(batsman,bowler,score):
    l = [0, 1, 6, 4, 'W', 'WD']
    score = random.choice(l)
    raw_commentary=''
    if score == 0:
        str_model_norun = '../commentary model/Models/Model_NoRun'
        raw_commentary= GenerateCommentarybasedonEvent(str_model_norun, "No runs scored" )
    elif score == 6:
        str_model_six = '../commentary model/Models/Model_Six'
        raw_commentary= GenerateCommentarybasedonEvent(str_model_six, f'{batsman} hits a six' )
    elif score == 4:
        str_model_Four = '../commentary model/Models/Model_Four'
        raw_commentary= GenerateCommentarybasedonEvent(str_model_Four, f'{batsman} hits a four' )
    elif score == 1:
        str_model_One = '../commentary model/Models/Model_One'
        raw_commentary= GenerateCommentarybasedonEvent(str_model_One, f'{batsman} hits a 1 run,' )
    elif score =='W':
        str_model_One = '../commentary model/Models/Model_Out'
        raw_commentary= GenerateCommentarybasedonEvent(str_model_One, f'{batsman} is out' )
    else:
        print("unknown")

    commentary = raw_commentary.replace("{{BATSMAN}}",batsman).replace("{{BOWLER}}",bowler)
    return commentary

In [9]:

model_load_path = os.path.abspath('./models')
test_video_path = 'New video.mp4'
video_summary_save_path = os.path.abspath('./results_summary')
tesseract_path = f'C:\Program Files\Tesseract-OCR\\tesseract.exe'
start_time = time.time()

# Set the starting position

#Path for the video 
vidcap = cv2.VideoCapture(f'{test_video_path}')
vidcap.set(cv2.CAP_PROP_POS_MSEC, 120000)
pytesseract.pytesseract.tesseract_cmd = tesseract_path
frame_count = 0
bats1_x,bats1_y,bats1_w,bats1_h = 650,925,430,50 # Batsman1 boundaries
bats2_x,bats2_y,bats2_w,bats2_h = 650,990,430,50 # Batsman2 boundaries
bowl_x,bowl_y,bowl_w,bowl_h = 1275,925,430,50  # Bowler boundaries
score_x,score_y,score_w,score_h = 1275,985,420,50 # Score boundaries
while (True):
    success, img = vidcap.read()
    
    if success:
        cv2.imwrite(f'frames/frame_{frame_count}.jpg', img)
        frame_count += 1
        batsman_1 =extract_text_from_roi(frame_count, bats1_x, bats1_y, bats1_w, bats1_h, img)
        batsman_2 = extract_text_from_roi(frame_count, bats2_x, bats2_y, bats2_w, bats2_h, img)
        bowler =extract_text_from_roi(frame_count, bowl_x, bowl_y, bowl_w, bowl_h, img)
        print("Output Bats1",batsman_1)
        print("Output Bats2",batsman_2)
        print("Output bowl",bowler)
        score = getScore(score_x, score_y, score_w, score_h, img)
        print("Score",score)
        current_batsman =''
        if batsman_1.__contains__('#') or batsman_1.__contains__('¢'):
            current_batsman = batsman_1.replace('#','').replace('¢','')
        elif batsman_2.__contains__('#') or batsman_2.__contains__('¢'):
            current_batsman = batsman_2.replace('#','').replace('¢','')
        else:
            current_batsman=''
        
        if current_batsman=='':
            print("No commentary required")
        else:
            commentary= generateCommentary(current_batsman,bowler,score)
            print("Generated Commentary:",commentary)
        cv2.rectangle(img, (bowl_x, bowl_y), (bowl_x+bowl_w, bowl_y+bowl_h), (0, 20, 255), 2)
        cv2.rectangle(img, (bats1_x, bats1_y), (bats1_x+bats1_w, bats1_y+bats1_h), (255, 20, 60), 2)
        cv2.rectangle(img, (bats2_x, bats2_y), (bats2_x+bats2_w, bats2_y+bats2_h), (0, 255, 0), 2) 
        cv2.rectangle(img, (score_x, score_y), (score_x+score_w, score_y+score_h), (255, 255, 255), 2)
        cv2.imshow('frame', img)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break   
    else:
        break 
      

cv2.destroyAllWindows()

print('--- %s seconds ---' % (time.time() - start_time))




Output Bats1 RAHANE 
Output Bats2 # GAIKWAD 
Output bowl ARSHAD - . |
wo1 1woel 1

Score wo1 1woel 1



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0:  GAIKWAD  hits a 1 run, and then his 2nd hit of the season.

"He got a 2nd and a 2," White said. "That's an awesome thing to say. He's going to be hitting a lot of balls. He's going to be playing right away, and it's going to take him a few years to get it to the plate.

"We are going to try to play the game of catch."

Alf
Generated Commentary: . GAIKWAD  hits a 1 run, and then his 2nd hit of the season.

"He got a 2nd and a 2," White said. "That's an awesome thing to say. He's going to be hitting a lot of balls. He's going to be playing right away, and it's going to take him a few years to get it to the plate.

"We are going to try to play the game of catch."

Alf
Output Bats1 RAHANE 
Output Bats2 # GAIKWAD 
Output bowl ARSHAD - . |
wo 1 1Wwoe]1 1

Score wo 1 1Wwoe]1 1



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0:  GAIKWAD  hits a 1 run, which will give us a two-run lead and we start to move forward. It's still early in the game and this is going to be one of those days where it's just one swing, a one-hit game and you have to do something about it. It doesn't work that way in the game. It's all good, but you have to be creative and keep it coming. It's just a different team at the moment."
Generated Commentary: . GAIKWAD  hits a 1 run, which will give us a two-run lead and we start to move forward. It's still early in the game and this is going to be one of those days where it's just one swing, a one-hit game and you have to do something about it. It doesn't work that way in the game. It's all good, but you have to be creative and keep it coming. It's just a different team at the moment."
Output Bats1 RAHANE 
Output Bats2 # GAIKWAD 
Output bowl ARSHAD - . |
wo 11Wwoe]1 1

Score wo 11W

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0:  GAIKWAD  hits a six over from Atsu over long-on. It was a low-handed sweep and he got away with it with ease
{{BOWLER}} to {{BATSMAN}}, S Volkswagen, that is the ball, that is the ball! You have seen the ball! How much power does this ball have when it has been given to him by his batsman, and now it is hitting the batsman with it? It is unbelievable. It
Generated Commentary: . GAIKWAD  hits a six over from Atsu over long-on. It was a low-handed sweep and he got away with it with ease
{{BOWLER}} to {{BATSMAN}}, S Volkswagen, that is the ball, that is the ball! You have seen the ball! How much power does this ball have when it has been given to him by his batsman, and now it is hitting the batsman with it? It is unbelievable. It
Output Bats1 RAHANE 
Output Bats2 # GAIKWAD 
Output bowl ARSHAD - .
wo 1 1Wwoe1 1

Score wo 1 1Wwoe1 1



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0:  GAIKWAD  hits a 1 run, and B.H.W. gets to it. B.H.W. walks to the right of the line. He runs a single up the middle that falls to the right side of the field, which B.H.W. has to scramble down, but the catcher can't get him to the catcher's feet. As he continues to throw the ball, it hits the catcher in the throat, and his backhands bounce up and
Generated Commentary: . GAIKWAD  hits a 1 run, and B.H.W. gets to it. B.H.W. walks to the right of the line. He runs a single up the middle that falls to the right side of the field, which B.H.W. has to scramble down, but the catcher can't get him to the catcher's feet. As he continues to throw the ball, it hits the catcher in the throat, and his backhands bounce up and
Output Bats1 RAHANE 
Output Bats2 # GAIKWAD 
Output bowl ARSHAD - .
wo11Wwoe]1 1

Score wo11Wwoe]1 1

unknown
Generated Commentary: 
Output Bats1 RAHANE 
Output Bats

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\naveen.muralidharan\AppData\Local\Temp\ipykernel_3564\1336139700.py:43 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\naveen.muralidharan\\AppData\\Local\\Temp\\ipykernel_3564\\1336139700.py'            │
│                                                                                                  │
│ C:\Users\naveen.muralidharan\AppData\Local\Temp\ipykernel_3564\3289721542.py:13 in               │
│ generateCommentary                                                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\naveen.muralidharan\\AppData\\Local\\Temp\\ipykernel_3564\\3289721542.py'            │
│                                                                                                  │
│ C:\Users\naveen.muralidharan\AppData\Local\Temp\ipykernel_3564\3695080567.py:3 in                │
│ GenerateCommentarybasedonEvent                                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\naveen.muralidharan\\AppData\\Local\\Temp\\ipykernel_3564\\3695080567.py'            │
│                                                                                                  │
│ C:\Users\naveen.muralidharan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kf │
│ ra8p0\LocalCache\local-packages\Python39\site-packages\transformers\modeling_utils.py:2405 in    │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2402 │   │   │   │   │   │   " to load this model from those weights."                         │
│   2403 │   │   │   │   │   )                                                                     │
│   2404 │   │   │   │   else:                                                                     │
│ ❱ 2405 │   │   │   │   │   raise EnvironmentError(                                               │
│   2406 │   │   │   │   │   │   f"Error no file named {_add_variant(WEIGHTS_NAME, variant)}, {TF  │
│   2407 │   │   │   │   │   │   f" {TF_WEIGHTS_NAME + '.index'} or {FLAX_WEIGHTS_NAME} found in   │
│   2408 │   │   │   │   │   │   f" {pretrained_model_name_or_path}."                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in 
directory ../commentary model/Models/Model_Four.

In [10]:
"""
# Load image, grayscale, Otsu's threshold
image = cv2.imread('temp\extracted_1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)[1]

# Find contours, sort from left-to-right, then crop
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts, _ = contours.sort_contours(cnts, method="left-to-right")

# Filter using contour area and extract ROI
ROI_number = 0
for c in cnts:
    area = cv2.contourArea(c)
    if area > 10:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
        ROI_number += 1

cv2.imshow('thresh', thresh)
cv2.imshow('image', image)
cv2.waitKey()"""

'\n# Load image, grayscale, Otsu\'s threshold\nimage = cv2.imread(\'temp\\extracted_1.jpg\')\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\nthresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV)[1]\n\n# Find contours, sort from left-to-right, then crop\ncnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\ncnts = cnts[0] if len(cnts) == 2 else cnts[1]\ncnts, _ = contours.sort_contours(cnts, method="left-to-right")\n\n# Filter using contour area and extract ROI\nROI_number = 0\nfor c in cnts:\n    area = cv2.contourArea(c)\n    if area > 10:\n        x,y,w,h = cv2.boundingRect(c)\n        ROI = image[y:y+h, x:x+w]\n        cv2.imwrite(\'ROI_{}.png\'.format(ROI_number), ROI)\n        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)\n        ROI_number += 1\n\ncv2.imshow(\'thresh\', thresh)\ncv2.imshow(\'image\', image)\ncv2.waitKey()'

: 